## Data Preparation - Tokenize FAQ dataset using custom GPT2 tokenizer

##### Prerequisites

In [ ]:
%%capture

!pip install --upgrade jupyter
!pip install --upgrade ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
%%capture

!pip install transformers==4.18.0
!pip install datasets==2.9.0
!pip install pandas==1.4.1
!pip install numpy==1.22.2
!pip install torch==1.8.1

#### Imports 

In [2]:
from transformers import GPT2Tokenizer
from transformers import set_seed
from datasets import load_dataset
from datasets import DatasetDict
import transformers
import pandas as pd
import numpy as np
import datasets 
import logging
import torch
import os

##### Setup logging

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [4]:
logger.info(f'[Using transformers version: {transformers.__version__}]')
logger.info(f'[Using datasets version: {datasets.__version__}]')
logger.info(f'[Using torch version: {torch.__version__}]')
logger.info(f'[Using pandas version: {pd.__version__}]')
logger.info(f'[Using numpy version: {np.__version__}]')

[Using transformers version: 4.18.0]
[Using datasets version: 2.9.0]
[Using torch version: 1.8.1+cu102]
[Using pandas version: 1.4.1]
[Using numpy version: 1.22.2]


#### Setup essentials 

In [5]:
np.random.seed(123)
set_seed(123)

In [6]:
BOS_TOKEN = '<|startoftext|>'
EOS_TOKEN = '<|endoftext|>'
PAD_TOKEN = '<|pad|>'

In [7]:
MAX_LEN = 512
N_GPUS = 1
num_proc = int(os.cpu_count()/N_GPUS)
num_proc

4

#### Load FAQ dataset 

In [8]:
data = load_dataset('csv', 
                     data_files='./data/faq_train.csv', 
                     column_names=['question', 'answer'], 
                     delimiter=',', 
                     split='train', 
                     #cache_dir='/tmp/cache',
                     download_mode='force_redownload')

Using custom data configuration default-a2711870881a3b6a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a2711870881a3b6a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [9]:
logger.info(f'Loaded dataset: {data}')

Loaded dataset: Dataset({
    features: ['question', 'answer'],
    num_rows: 1587
})


#### Create data splits 

In [10]:
train_validation_test = data.train_test_split(shuffle=True, seed=123, test_size=0.1)
data_splits = DatasetDict({'train': train_validation_test['train'],  
                           'validation': train_validation_test['test']})
logger.info(f'Data splits: {data_splits}')

Data splits: DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1428
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 159
    })
})


#### Setup the custom GPT2 tokenizer 

Re-create the custom tokenizer we created in the previous medium article "[Easily Build Your Own GPT from Scratch using AWS: A Comprehensive Guide for Domain Adaptation](https://medium.com/@shankar.arunp/easily-build-your-own-gpt-from-scratch-using-aws-51811b6355d3)"

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained('./vocab-custom', 
                                          bos_token=BOS_TOKEN,
                                          eos_token=EOS_TOKEN, 
                                          pad_token=PAD_TOKEN, 
                                          lower=True,
                                          return_tensors='pt')
tokenizer.padding_side = 'left'
tokenizer.model_max_length = MAX_LEN
logger.info(f'Tokenizer: {tokenizer}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Tokenizer: PreTrainedTokenizer(name_or_path='./vocab-custom', vocab_size=50257, model_max_len=512, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|pad|>'})


#### Tokenize data splits 

In [12]:
def tokenize(samples: list):
    questions = samples['question']
    answers = samples['answer']
    logger.info(f'Tokenizing QA pairs of length = {len(questions)}')
    
    input_ids = []
    attention_mask = []
    labels = []
    
    for question, answer in zip(questions, answers):
        prompted_input = f'{BOS_TOKEN}question: {question}{PAD_TOKEN}answer: {answer}{EOS_TOKEN}'
        tokenized_input = tokenizer(prompted_input, 
                                    truncation=True, 
                                    max_length=MAX_LEN, 
                                    padding='max_length')
        input_ids.append(torch.tensor(tokenized_input['input_ids'], dtype=torch.long))
        attention_mask.append(torch.tensor(tokenized_input['attention_mask']))
        labels.append(torch.tensor(tokenized_input['input_ids']))

    return {'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels}

In [13]:
tokenized_data = data_splits.map(tokenize, batched=True, 
                                 num_proc=num_proc, 
                                 load_from_cache_file=False,
                                 remove_columns=['question', 'answer'])
tokenized_data.set_format('pt', 
                          columns=['input_ids', 'attention_mask', 'labels'], 
                          output_all_columns=True)
logger.info(f'Tokenized data = {tokenized_data}')

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

Tokenizing QA pairs of length = 357


#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

Tokenizing QA pairs of length = 357
Tokenizing QA pairs of length = 357


...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Tokenizing QA pairs of length = 39


#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

Tokenizing QA pairs of length = 40
Tokenizing QA pairs of length = 40
Tokenized data = DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1428
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 159
    })
})


#### Save tokenized data splits to local

In [14]:
tokenized_data.save_to_disk('./data/tokenized-custom')

Saving the dataset (0/1 shards):   0%|          | 0/1428 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/159 [00:00<?, ? examples/s]